<a href="https://colab.research.google.com/github/comb0601/Kaggle-Notebooks/blob/main/deepfake_1_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mtcnn
!pip install facenet-pytorch

import os
import glob
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from skimage import measure
from facenet_pytorch import MTCNN
from PIL import Image

     |████████████████████████████████| 2.3MB 5.6MB/s 
     |████████████████████████████████| 1.9MB 5.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#TEST_PATH = '/content/'
TRAIN_PATH = '/gdrive/My Drive/'

#metadata = '../input/deepfake-detection-challenge/train_sample_videos/metadata.json'

In [ ]:
# load the filenames for train videos
train_fns = sorted(glob.glob(TRAIN_PATH + '*.mp4'))

# load the filenames for test videos
#test_fns = sorted(glob.glob(TEST_PATH + '*.mp4'))

print('There are {} samples in the train set.'.format(len(train_fns)))
#print('There are {} samples in the test set.'.format(len(test_fns)))

There are 2 samples in the train set.


In [ ]:
train_fns

['/gdrive/My Drive/fake.mp4', '/gdrive/My Drive/real.mp4']

In [ ]:
def get_frames(filename):
    '''
    Get all frames from the video
    INPUT:
        filename - video filename
    OUTPUT:
        frames - the array of video frames
    '''
    frames = []
    
    cap = cv2.VideoCapture(filename)
    
    #while(cap.isOpened()):
    while len(frames) < 301 :
        ret, frame = cap.read()
                
        if not ret:
            break;
            
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(image)

    cap.release()
    cv2.destroyAllWindows()
    return frames



In [ ]:
# 얼굴 감지
# Create face detector
for video in train_fns[:1] :
  frames = get_frames(video)
  mtcnn = MTCNN(margin=20, keep_all=False, post_process=False,device='cuda:0')  #keep_all : multiple faces in a single image,device='cuda:0'

  
  save_paths = [str(video)+f'image_{i}.jpg' for i in range(len(frames[:100]))]
  mtcnn(frames[:100], save_path=save_paths)

  save_paths = [str(video)+f'image_{i}.jpg' for i in range(len(frames[100:200]))]
  mtcnn(frames[100:200], save_path=save_paths)

  save_paths = [str(video)+f'image_{i}.jpg' for i in range(len(frames[200:300]))]
  mtcnn(frames[200:300], save_path=save_paths)

# 새 섹션

In [ ]:
def calcRGBCenter(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #RGB 형태로 변환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    k = 1
    clt = KMeans(n_clusters = k)
    clt.fit(image)

    return clt.cluster_centers_[0]


def calcHSVCenter(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) #HSV 형태로 변환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    k = 1
    clt = KMeans(n_clusters = k)
    clt.fit(image)

    return clt.cluster_centers_[0]

In [ ]:
def get_image_difference(image_1, image_2):  # 히스토그램 차이
    first_image_hist = cv2.calcHist([image_1], [0], None, [256], [0, 256])
    second_image_hist = cv2.calcHist([image_2], [0], None, [256], [0, 256])

    img_hist_diff = cv2.compareHist(first_image_hist, second_image_hist, cv2.HISTCMP_BHATTACHARYYA)
    #img_template_probability_match = cv2.matchTemplate(first_image_hist, second_image_hist, cv2.TM_CCOEFF_NORMED)[0][0]
    #img_template_diff = 1 - img_template_probability_match

    # taking only 10% of histogram diff, since it's less accurate than template method
    #commutative_image_diff = (img_hist_diff / 10) + img_template_diff
    return img_hist_diff

In [ ]:
train_fns[0] + 'image_1.jpg'

'/gdrive/My Drive/fake.mp4image_1.jpg'

In [ ]:
videoname

'fake.mp4'

In [ ]:
    #for videoname in train_fns : # 하나의 동영상 이름 가져오기
    videoname = train_fns[0]
    frames = []
    is_fake = True
    #if videoname.startswith("fake"):
    #     is_fake = True

    for i in range(100) :      # 가져온 동영상 얼굴 사진들로 프레임 생성
      frame = cv2.imread(videoname+'image_'+str(i)+'.jpg')
      frames.append(frame)

    resfile = open(videoname+"_result.csv", "a")
    resfile.write("mse,psnr,ssim,hist_diff,r_diff,g_diff,b_diff,h_diff,s_diff,v_diff,deepfake\n")

    for i in range(len(frames)-1) :
        img1 = frames[i] # 첫 프레임
        img2 = frames[i+1] #다음 프레임

        mse = np.sum((img1.astype("float") - img2.astype("float")) ** 2) #MSE
        mse /= float(img1.shape[0] * img1.shape[1])

        psnr = cv2.PSNR(img1, img2) # 프레임 간 PSNR값

        ssim = measure.compare_ssim(img1, img2,multichannel=True) # SSIM 멀티채널 오픈필요

        histogram_diff = get_image_difference(img1,img2) # 프레임간 히스토그램 차이

        rgb_diff = abs(calcRGBCenter(img2) - calcRGBCenter(img1))

        hsv_diff = abs(calcHSVCenter(img2) - calcHSVCenter(img1))

        resfile.write(
            str(mse) + ',' + str(psnr) + ',' + str(ssim) + ',' + str(histogram_diff) + ',' +
            str(rgb_diff[0]) + ',' + str(rgb_diff[1]) + ',' + str(rgb_diff[2]) + ',' +
            str(hsv_diff[0]) + ',' + str(hsv_diff[1]) + ',' + str(hsv_diff[2]) + ',' + str(is_fake) + '\n'
        )



AttributeError: ignored